In [4]:
import pandas as pd
import numpy as numpy
import datetime
from tqdm.notebook import tqdm
from sklearn

class Results:
    @staticmethod
    def scrape(race_id_list):
        race_results = {}
        for race_id in tqdm(race_id_list):
            time.sleep(1)
        return